# Notebook: Split Dataset in folds

## Packages

In [18]:
from llama_cpp import Llama
import numpy as np
import itertools
import warnings
import random
import json

## Settings

In [2]:
SPLIT = 0

# Select Model for Synthesis
MODELS = ["Llama70B"]
MODEL = MODELS[0]

# Number of Examples to be generated with the LLM
N_SYNTH = 500

# Setup Classes/Polarities for Synthesis
CLASSES  = ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]
POLARITY = ["POSITIVE", "NEUTRAL", "NEGATIVE"]

## Parameters

In [16]:
DATASET_PATH = f'../03 dataset split/real/real_{SPLIT}.json'
MAX_TOKENS = 200
CONTEXT_SIZE = 2048
SEED = 43

In [4]:
random.seed(SEED)

## Code

### Create Labels for Synthesis

#### Calculate Number Examples with 1, 2 or 3 Aspects and total number of Aspects

In [5]:
def divide_equally(x):
    equally = x // 3
    remainder = x % 3
    values = [equally, equally, equally]

    for i in range(remainder):
        values[i] += 1

    random.shuffle(values)

    return values

n_for_n_aspects = divide_equally(500)
n_aspects_synth = sum([n_for_n_aspects[0], n_for_n_aspects[1] * 2, n_for_n_aspects[2] * 3])

In [6]:
n_aspects_synth, n_for_n_aspects

(1001, [166, 167, 167])

#### Generate Random Tuples

Each combination of aspect-category + polarity is equally frequent

In [7]:
combinations = list(itertools.product(CLASSES, POLARITY))
total_combinations = len(combinations)
desired_count_per_combination = n_aspects_synth // total_combinations
remaining_tuples = n_aspects_synth % total_combinations
tuples = []
for combination in combinations:
    for _ in range(desired_count_per_combination):
        tuples.append(combination)

if remaining_tuples > 0:
    tuples.extend(combinations[:remaining_tuples])
    
random.shuffle(tuples)

In [8]:
len(tuples), tuples[:10]

(1001,
 [('GENERAL_IMPRESSION', 'POSITIVE'),
  ('SERVICE', 'POSITIVE'),
  ('SERVICE', 'POSITIVE'),
  ('FOOD', 'POSITIVE'),
  ('SERVICE', 'NEGATIVE'),
  ('SERVICE', 'NEGATIVE'),
  ('PRICE', 'NEGATIVE'),
  ('SERVICE', 'POSITIVE'),
  ('GENERAL_IMPRESSION', 'NEGATIVE'),
  ('SERVICE', 'NEUTRAL')])

#### Generate Labels

In [9]:
n_aspects = 1
labels = []
idx_aspects = 0

for k in n_for_n_aspects:
    for n in range(k):
        label = []
        for aspect_idx in range(n_aspects):
            label.append(tuples[idx_aspects])
            idx_aspects += 1
        labels.append(label)
        #print(k, n, n_aspects, label)
    n_aspects += 1

In [10]:
random.shuffle(labels)

### Setup Prompt Template

In [11]:
with open('../prompt_template.txt', 'r') as file:
    PROMPT_TEMPLATE = file.read()

In [12]:
print(PROMPT_TEMPLATE)

Bitte erzeuge genau einen Satz einer Restaurantbewertung, die für das Training eines Modells für die Aspekt-basierte Sentiment Analyse verwendet werden kann.
Gegeben ist ein Label in Form eines Arrays, wobei für jedes Sentiment, mit dem eine Aspekt-Kategorie im Satz adressiert wird, ein Tuple (Aspekt-Kategorie, Aspekt-Sentiment) vorhanden ist.
Eine Aspekt-Kategorie kann mehrfach im Satz adressiert werden, auch mit verschiedenen Sentiment-Polaritäten.

* Aspekt-Kategorien hat nur diese Werte: ["GENERAL_IMPRESSION", "FOOD", "SERVICE", "PRICE", "AMBIENCE"]
* Sentiment nur diese Polaritäten: ["negative", "neutral", "positive"]

Auf Basis des Arrays soll genau ein deutscher Satz erzeugt werden, der die in den Tuple definierten Aspekte enthält.
Zusätzlich kann für ein im Label vorgegebenes Tuple (Aspekt-Kategorie, Aspekt-Sentiment) ein Aspekte-Term im Text vorliegen.
Der Aspekt-Term wird mithilfe eines xml-Tags markiert.

Gebe nur die Prediction zurück, ohne Kommentare oder zusätzlichen Text

### Get Examples

#### Load Dataset

In [19]:
with open(DATASET_PATH, 'r', encoding='utf-8') as json_file:
    dataset = json.load(json_file)

#### Get Example as Text

In [21]:
dataset[:1]

[{'tags': [{'end': 9,
    'start': 0,
    'tag_with_polarity': 'FOOD-POSITIVE',
    'tag_with_polarity_and_type': 'FOOD-POSITIVE-explicit',
    'text': 'Schnitzel',
    'type': 'label-explicit',
    'label': 'FOOD',
    'polarity': 'POSITIVE'}],
  'text': 'Schnitzel top.',
  'aspect_available_without_judgement': False,
  'two_or_more_sentences': False,
  'city': 'münchen',
  'date': '2022-08-20',
  'title': 'Schönes Ambiente',
  'rating': 4.0,
  'review_id': 855174268,
  'page_index': 3,
  'author_name': 'Flocke24',
  'sentence_idx': 8,
  'language_code': 'de',
  'restaurant_id': 807825,
  'author_location': 'Bad Laasphe, Deutschland',
  'restaurant_name': 'Augustiner-Keller',
  'detected_language': 'de',
  'text_noanonymization': 'Biergarten strahlt eine gemütliche Atmosphäre. Bedienungen sind sehr auf Zack und trotz der Menschenmassen steht’s freundlich. Der Saal innen erinnert an die große Halle von Harry Potter. Hier muss man leider sagen, dass man sich wie in einer Bahnhofshalle v

In [58]:
def convert_ner_to_xml(ner_dict):
    text = ner_dict['text']
    tags = ner_dict['tags']
    tag_positions = [] 

    for tag in tags:
        start = tag['start']
        end = tag['end']
        label = tag['label']
        polarity = tag['polarity']
        tag_type = tag['type']

        # Füge das Tag nur ein, wenn es sich um "label-explicit" handelt
        if tag_type == 'label-explicit':
            tag_positions.append((start, f'<aspect-term aspect="{label}" polarity="{polarity}">'))
            tag_positions.append((end, '</aspect-term>'))

    tag_positions.sort(reverse=True, key=lambda x: x[0])

    xml_text = list(text)
    for position, tag in tag_positions:
        xml_text.insert(position, tag)

    return ''.join(xml_text)

In [67]:
dataset[:3]

[{'tags': [{'end': 9,
    'start': 0,
    'tag_with_polarity': 'FOOD-POSITIVE',
    'tag_with_polarity_and_type': 'FOOD-POSITIVE-explicit',
    'text': 'Schnitzel',
    'type': 'label-explicit',
    'label': 'FOOD',
    'polarity': 'POSITIVE'}],
  'text': 'Schnitzel top.',
  'aspect_available_without_judgement': False,
  'two_or_more_sentences': False,
  'city': 'münchen',
  'date': '2022-08-20',
  'title': 'Schönes Ambiente',
  'rating': 4.0,
  'review_id': 855174268,
  'page_index': 3,
  'author_name': 'Flocke24',
  'sentence_idx': 8,
  'language_code': 'de',
  'restaurant_id': 807825,
  'author_location': 'Bad Laasphe, Deutschland',
  'restaurant_name': 'Augustiner-Keller',
  'detected_language': 'de',
  'text_noanonymization': 'Biergarten strahlt eine gemütliche Atmosphäre. Bedienungen sind sehr auf Zack und trotz der Menschenmassen steht’s freundlich. Der Saal innen erinnert an die große Halle von Harry Potter. Hier muss man leider sagen, dass man sich wie in einer Bahnhofshalle v

In [77]:
def get_examples_for_aspects_in_label(unique_aspects):
    return [random.choice([entry for entry in dataset if any(tag['label'] == aspect for tag in entry['tags'])]) for aspect in unique_aspects]
    

In [88]:
ex = get_examples_for_aspects_in_label(["FOOD", "SERVICE"])
len(ex), ex

(2,
 [{'tags': [{'end': 17,
     'start': 12,
     'tag_with_polarity': 'PRICE-NEGATIVE',
     'tag_with_polarity_and_type': 'PRICE-NEGATIVE-explicit',
     'text': 'Preis',
     'type': 'label-explicit',
     'label': 'PRICE',
     'polarity': 'NEGATIVE'},
    {'end': 53,
     'start': 48,
     'tag_with_polarity': 'FOOD-NEGATIVE',
     'tag_with_polarity_and_type': 'FOOD-NEGATIVE-explicit',
     'text': 'Essen',
     'type': 'label-explicit',
     'label': 'FOOD',
     'polarity': 'NEGATIVE'}],
   'text': 'Ein stolzer Preis sollte sich mit einem Stolzen Essen decken!',
   'aspect_available_without_judgement': False,
   'two_or_more_sentences': False,
   'city': 'hamburg',
   'date': '2023-02-08',
   'title': 'Luft nach oben',
   'rating': 2.0,
   'review_id': 877947227,
   'page_index': 6,
   'author_name': 'O5223WUlydiah',
   'sentence_idx': 4,
   'language_code': 'de',
   'restaurant_id': 1483463,
   'author_location': 'Hamburg, Deutschland',
   'restaurant_name': 'Bullerei',
   'd

### Create Synthetic Samples

In [68]:
for label in labels[:2]:
    unique_aspects = list(set(aspect for aspect, _ in label))
    sample_examples = {aspect: next((review['text'] for review in dataset if any(tag['label'] == aspect for tag in review['tags'])), None) for aspect in unique_aspects}
    print(sample_examples)
    prompt_footer = f'\nLabel: {str(label)}\nPrediction:'
    prompt = PROMPT_TEMPLATE + prompt_footer
    #print(prompt)

{'SERVICE': 'Das Personal freundlich.', 'PRICE': 'Ein stolzer Preis sollte sich mit einem Stolzen Essen decken!', 'FOOD': 'Schnitzel top.'}
{'GENERAL_IMPRESSION': None, 'PRICE': 'Ein stolzer Preis sollte sich mit einem Stolzen Essen decken!'}


### Setup Model

In [22]:
llm = Llama(model_path="llama-2-13b.Q4_0.gguf", seed=SEED, n_gpu_layers=1, n_ctx=CONTEXT_SIZE, verbose=False)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from llama-2-13b.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama

In [27]:
prompt = "Test"

In [28]:
output = llm(prompt, max_tokens=MAX_TOKENS, stop=["Q:", "\n"], echo=True)
print(output)

{'id': 'cmpl-f1238d19-4f7a-4aa0-8229-7811a49e3540', 'object': 'text_completion', 'created': 1695027955, 'model': 'llama-2-13b.Q4_0.gguf', 'choices': [{'text': 'Die FDP ist eine Partei, die auf den Verhaltenskodex der FDP Deutschland verpflichtet wurde. In allen Regionen werden wir auf Grundlage von Vorschriften des jeweiligen Landes zertifiziert und regelmäßig aufrecht gehalten.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 9, 'completion_tokens': 50, 'total_tokens': 59}}
